# OpendTect Python Bindings

Release 6.6.7 of the wmPlugins suite includes experimental Python bindings to OpendTect. There are a number of limitations to be aware of:
-  Currently the bindings only provide access to information about Surveys and Wells
-  The bindings have been built against Python 3.7 and may/may not work with other versions of Python
-  The bindings can't be used with a Python IDE that depends on a different Qt version (5.15.1) to OpendTect. This rules out the spyder IDE in the OpendTect Python environments which use Qt 5.9.6
-  Jupyter Lab or Notebooks, however, work really well 

## Installation
The recommended procedure for installation is to use the OpendTect installation manager to install the wmPlugins. This will install the library containing the bindings in the OpendTect bin/"Platform"/Release folder where "Platform" will either be win64 for Windows or lux64 for Linux.
    
You also need a Python environment with at least the following packages: 
-  Pandas
-  Shapely and Geopandas
    
The OpendTect Python environments don't include Shapely or Geopandas so your options are to use conda to install those dependencies or set up your own custom Python environment. 
    
## Setting the PYTHONPATH and Python Environment
The folder containing the bindings library must be on the PYTHONPATH to be found by an import statement in a Python script. You can use the OpendTect Python Settings dialog (accessible from the Utilities| Installation|Python Settings application menu) and add the location of the OpendTect executable files to the Custom Module Path. This dialog is also where you can select a custom Python environment to use and also add an icon to the OpendTect toolbar to start your chosen IDE from the specified Custom Python environment with the custom module path added to the PYTHONPATH.

![](images/python_settings.jpg)
    
Alternatively, the PYTHONPATH can be modified in the script or a notebook cell before the actual import.

import sys
sys.path.insert(0,r'C:\Program Files\OpendTect\6.6.0\bin\win64\Release')

## wmodpy Module
The module contains a function (get_surveys)that lists the OpendTect surveys in a user supplied base data folder and classes to access:
-  Survey information (Survey class)
-  Well data (Wells class)

If you are working in a Jupyter notebook or Python command shell you can use the help() function to get more information about a class or function:

In [5]:
import wmodpy
help(wmodpy.get_surveys)

Help on built-in function get_surveys in module wmodpy:

get_surveys(...) method of builtins.PyCapsule instance
    get_surveys(survey_data_root: str) -> list
    
    Return list of survey names in given survey data root



### wmodpy.get_surveys()
Give it the location of a base data folder and you get back a python list of the contained projects/surveys.

In [16]:
wmodpy.get_surveys('Y:\CooperBasin\ODData')


['13CP06_Dundinna', 'Cooper2D']

### wmodpy.Survey Class
This class provides some basic information about an OpendTect project/survey. Creating a Survey object requires both the base data folder location and the project/survey name. The other data specific classes require a Survey object for context.

In [15]:
f3demo = wmodpy.Survey('Y:\ODData', 'F3_Demo_2020')
f3demo.name()

'F3 Demo 2020'

In [17]:
f3demo.has3d()

True

In [18]:
f3demo.has2d()

True

In [19]:
f3demo.epsg() 

'EPSG:23031'

It's easy to work with multiple projects/surveys at the same time:

In [20]:
penobscot = wmodpy.Survey('Y:\ODData', 'Penobscot')
penobscot.epsg()

'EPSG:2294'

In [21]:
f3demo.epsg()

'EPSG:23031'

### wmodpy.Wells Class
This class provides access to well data within an OpendTect project/survey. The class constructor requires a Survey object.
The get_well_names() method provides a list of well names in the project/survey.

In [22]:
f3wells = wmodpy.Wells(f3demo)
f3wells.get_well_names()

['F02-1', 'F03-2', 'F03-4', 'F06-1']

General well information is available as a python dictionary. 

In [23]:
f3wells.get_well_info()

{'Name': ['F02-1', 'F03-2', 'F03-4', 'F06-1'],
 'UWID': ['', '', '', ''],
 'State': ['', '', '', ''],
 'County': ['', '', '', ''],
 'WellType': ['none', 'none', 'none', 'none'],
 'X': [606554.0, 619101.0, 623255.98, 607903.0],
 'Y': [6080126.0, 6089491.0, 6082586.87, 6077213.0],
 'ReplacementVelocity': [2000.0, 2000.0, 2000.0, 2000.0],
 'GroundElevation': [1.0000000150474662e+30,
  1.0000000150474662e+30,
  1.0000000150474662e+30,
  1.0000000150474662e+30]}

In [24]:
wmodpy.Wells(penobscot).get_well_names()

['L-30', 'B-41']

In [25]:
wmodpy.Wells(penobscot).get_well_info()

{'Name': ['L-30', 'B-41'],
 'UWID': ['', ''],
 'State': ['', ''],
 'County': ['', ''],
 'WellType': ['none', 'none'],
 'X': [734338.8571, 731133.461],
 'Y': [4893837.87, 4894307.877],
 'ReplacementVelocity': [2000.0, 2000.0],
 'GroundElevation': [1.0000000150474662e+30, 1.0000000150474662e+30]}

If your Python environment includes Pandas there is a function with the same name suffixed by "_df" which will return the same information directly in a Pandas DataFrame

In [27]:
f3wells.get_well_info_df()

,Name,UWID,State,County,WellType,X,Y,ReplacementVelocity,GroundElevation
0,F02-1,,,,none,606554.00,6080126.00,2000.0,1.000000e+30
1,F03-2,,,,none,619101.00,6089491.00,2000.0,1.000000e+30
2,F03-4,,,,none,623255.98,6082586.87,2000.0,1.000000e+30
3,F06-1,,,,none,607903.00,6077213.00,2000.0,1.000000e+30


If your environment includes GeoPandas there is another function of the same name suffixed by "_gdf" which returns a GeoDataFrame with the well surface coordinates as Point geometries. The crs of the GeoDataFrame is set to the EPSG code of the survey object.

In [31]:
f3wells.get_well_info_gdf()

,Name,UWID,State,County,WellType,X,Y,ReplacementVelocity,GroundElevation,geometry
0,F02-1,,,,none,606554.00,6080126.00,2000.0,1.000000e+30,POINT (606554.000 6080126.000)
1,F03-2,,,,none,619101.00,6089491.00,2000.0,1.000000e+30,POINT (619101.000 6089491.000)
2,F03-4,,,,none,623255.98,6082586.87,2000.0,1.000000e+30,POINT (623255.980 6082586.870)
3,F06-1,,,,none,607903.00,6077213.00,2000.0,1.000000e+30,POINT (607903.000 6077213.000)


Likewise well log information for each well is also available as a python dictionary or as a Pandas dataframe 

In [28]:
f3wells.get_well_log_info_df('F02-1')

,Name,Mnem,Uom,DahRange,ValueRange
0,Density,RHOB,g/cc,"(305.1000061035156, 1424.1700439453125)","(1.9978599548339844, 2.237799882888794)"
1,Sonic,DT,us/ft,"(305.1000061035156, 1424.0999755859375)","(112.20144653320312, 176.78977966308594)"
2,Gamma Ray,GR,API,"(50.099998474121094, 1499.8499755859375)","(1.3392000198364258, 129.177001953125)"
3,Porosity,PHI,Fraction,"(305.1000061035156, 1424.1700439453125)","(0.25765201449394226, 0.40758100152015686)"
4,P-Impedance,IMP,Meter/second x Kg/m3,"(262.04998779296875, 1499.8399658203125)","(2846140.0, 5978060.0)"
5,P-Impedance_rel,IMP,Meter/second x Kg/m3,"(262.04998779296875, 1499.8399658203125)","(-477863.0, 410655.0)"
6,Vp,VEL,Meter/second,"(305.1000061035156, 1424.0999755859375)","(1724.3699951171875, 2714.530029296875)"
7,Vp_BLI,VEL,Meter/second,"(795.1500244140625, 1136.699951171875)","(2098.55712890625, 2567.929443359375)"
8,Vs_BLI,VEL,Meter/second,"(795.1500244140625, 1149.4500732421875)","(1009.1357421875, 1290.41943359375)"
9,Density_BLI,RHOB,g/cc,"(795.1500244140625, 1131.9000244140625)","(2.120549201965332, 2.260430335998535)"


And marker information for each well is also available as a python dictionary or Pandas dataframe depending on the function used.

In [29]:
f3wells.get_markers_df('F03-4')

,Name,Color,MD
0,Seasurface,#dc3232,30.000000
1,MFS11,#3232dc,479.739990
2,FS11,#32c832,515.341919
3,MFS10,#32c8c8,547.750000
4,MFS9,#ffd200,547.750000
5,MFS8,#dc00dc,547.750000
6,FS8,#00ff00,568.973572
7,FS7,#3232dc,583.972595
8,Truncation,#aa55ff,662.539917
9,Top Foresets,#ff0000,697.109985


As is the well track.

In [30]:
f3wells.get_track_df('F03-4')

,md,tvdss,x,y
0,0.0,-34.099998,623256.0000,6082587.0
1,100.0,65.900002,623256.0000,6082587.0
2,270.0,235.870056,623255.8750,6082590.0
3,380.0,345.859985,623255.8750,6082591.5
4,475.0,440.859985,623255.8125,6082591.5
5,655.0,620.859985,623255.8125,6082592.5
6,751.0,716.849976,623255.8125,6082593.5
7,856.0,821.840088,623255.7500,6082594.5
8,952.0,917.840088,623255.7500,6082595.5
9,1000.0,965.830078,623255.6875,6082596.5
